In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.coordinates as coord

from astropy.coordinates import SkyCoord
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014


from GaiaGalpy import gaia_to_galpy
from GaiaGalpy import Filter
from GaiaGalpy import Path


# Main source

Jo Bovy's lectures in galactice dynamics chapters 3,9,10,11

# Objective

Filter out the angular velocity of the stars that are in-tune with Sun first assuming a perfectly phi-symmetirc galactic disk and second taking average. They should be very similar since the time period that I am looking at is small compared to their orbital period. Allow a divergence of 20pc per 300Myr. First I need to tranfer to the galactocentric coordinate system using Astropy and then the rest would be easy. The stars must be in the vicinity of Sun therefore their radial distance from the center of the galaxy must be approximately equal to that of the Sun, a little up or down. The angular considerations are already taken into account in the first few sentences.

# Procedure

1. Define a function that tunrs data from Gaia into a proper formate so it can be used by Galpy–the should be in galactocentric coordinate. GaiaGalpy functions can be very useful.  
2. Define the Keplerian orbit for these stars disregarding any motion above the plane of the galaxy and apply the proper angular frequency and distance constraints to designate the candidate stars. 
3. Take into account the oscillatory frequencies in the motion of these stars and detect how close they would get to the Sun. 

# Thing to consider

I may want to take into account the spiral arms of the galaxy later on. What changes would occur if I consider the motion of these stars above the plane of the galaxy?

In [2]:
Init_Galpy= gaia_to_galpy('/Users/shasha/Desktop/Research/Manoj Kaplinghat /GaiaGalpy/Data/Gaia stars with radial velocity/GaiaSource_2851858288640_1584379458008952960.csv.gz')

In [3]:
Init_Galpy.shape

(453022, 7)

In [4]:
o= Orbit(Init_Galpy[:,1:],radec=True)
o0= Orbit(None)

In [5]:
def AngFreq(Init_Galpy):
    
    o= Orbit(Init_Galpy[:,1:],radec=True)
    o0= Orbit(None)

    W=[]

    for i in range(0, len(o)):

        R= o[i].R()
        vx= o[i].vx()
        vy= o[i].vy()
        S= (vx*vx + vy*vy)**0.5

        w= S/R

        W.append(w)

    a= Init_Galpy[:,0]
    a_list= list(a)

    df= pd.DataFrame(data=W)

    data= {'source_id': a_list ,'angular frequency': df.iloc[:,0]}
    D= pd.DataFrame(data=data, columns=['source_id','angular frequency'])


    return D

In [6]:
o= Orbit(Init_Galpy[:,1:],radec=True)
o0= Orbit(None)

W=[]

for i in range(0, len(o)):

    R= o[i].R()
    vx= o[i].vx()
    vy= o[i].vy()
    S= (vx*vx + vy*vy)**0.5

    w= S/R

    W.append(w)

a= Init_Galpy[:,0]
a_list= list(a)

In [7]:
aa= Init_Galpy[:,0].astype('int64')
aa_list= list(aa)

In [8]:
df= pd.DataFrame(data=W)

data= {'source_id': aa_list ,'angular frequency': df.iloc[:,0]}
D= pd.DataFrame(data=data, columns=['source_id','angular frequency'])


In [9]:
D

,source_id,angular frequency
0,15637976759168,26.793686
1,17149805245440,26.790641
2,21788369897344,21.550224
3,29278792888448,21.799226
4,33264522507136,17.617662
...,...,...
453017,1584370593196445696,31.350322
453018,1584371692708076544,24.182444
453019,1584376022035134464,32.477614
453020,1584378770814182656,24.695515


In [10]:
D.iloc[0,1]

26.79368635616854

In [11]:
D.iloc[1,1]

26.79064078417655

In [12]:
D.iloc[:,1]

0         26.793686
1         26.790641
2         21.550224
3         21.799226
4         17.617662
            ...    
453017    31.350322
453018    24.182444
453019    32.477614
453020    24.695515
453021    27.215887
Name: angular frequency, Length: 453022, dtype: float64

In [13]:
D.iloc[:,0]

0              15637976759168
1              17149805245440
2              21788369897344
3              29278792888448
4              33264522507136
                 ...         
453017    1584370593196445696
453018    1584371692708076544
453019    1584376022035134464
453020    1584378770814182656
453021    1584379458008952832
Name: source_id, Length: 453022, dtype: int64

In [14]:
df= pd.DataFrame(data=W, columns=['frequency'])

In [15]:
df.iloc[:,0]

0         26.793686
1         26.790641
2         21.550224
3         21.799226
4         17.617662
            ...    
453017    31.350322
453018    24.182444
453019    32.477614
453020    24.695515
453021    27.215887
Name: frequency, Length: 453022, dtype: float64

In [16]:
def FreqFil(data, b=0.001): #data should be a pandas data frame
    
    o0= Orbit(None)
    R0= o0.R()
    vx0= o0.vx()
    vy0= o0.vy()
    S0= (vx0*vx0 + vy0*vy0)**0.5
    w0= S0/R0
    
    d= np.array(data.iloc[:,1])
    
    WR=[] #This will give the relative error
    
    for i in range(0,len(d)):
        wr= ((d[i]-w0)**2)**0.5 /d[i]
        WR.append(wr)

    D0= {'source_id': data.iloc[:,0],'relative_frequency': WR}
    D1= pd.DataFrame(data=D0, columns=['source_id','relative_frequency'])
    
    D1_filtered= D1.query('relative_frequency < {0}'.format(b))
    
    return D1_filtered
    

In [17]:
FreqFil0= FreqFil(D,0.00001)

In [18]:
FreqFil0

,source_id,relative_frequency
18218,80443710957534848,3.702413e-07
54250,207734756224163328,8.122736e-06
61252,226831383412409088,7.438065e-06
70316,248545432394617344,1.029189e-06
78959,270387467096467584,7.371906e-06
86373,290773409108349184,3.784503e-06
109807,360232174274149120,7.785638e-06
150139,441538104214084736,5.979772e-07
171808,487052113165030528,2.112410e-06
172582,488746941619553280,6.220665e-06


In [19]:
FreqFil0.shape

(27, 2)

In [20]:
Filter=Filter('/Users/shasha/Desktop/Research/Manoj Kaplinghat /GaiaGalpy/Data/Gaia stars with radial velocity/GaiaSource_2851858288640_1584379458008952960.csv.gz')

In [21]:
Filter

,source_id,distance,radial_velocity,dec,ra,pmra,pmdec,rel_p_err,rel_rad_err,rel_dec_err,rel_pmdec_err,rel_pmra_err,rel_ra_err
8,15637976759168,0.762535,-29.915697,0.344149,45.194622,1.905093,-0.104026,0.020848,-0.127016,0.060397,-0.464400,0.023752,0.000478
11,17149805245440,0.613763,30.375573,0.403600,45.172876,8.397275,2.950902,0.027334,0.081721,0.086164,0.028881,0.009145,0.000788
12,21788369897344,0.481838,-2.647520,0.332088,44.872858,16.040961,-16.908529,0.021661,-3.108356,0.081807,-0.003521,0.003746,0.000846
15,29278792888448,0.767627,-12.840195,0.443261,45.149983,3.860539,-15.002175,0.028035,-0.056230,0.058889,-0.003970,0.016711,0.000747
16,33264522507136,0.704963,17.330323,0.473469,44.974661,10.434865,-24.788964,0.035740,0.044455,0.076573,-0.003193,0.007727,0.000963
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1584370593196445824,0.355762,24.199749,63.353966,183.098763,-30.092898,23.443970,0.007423,0.039228,0.000251,0.001324,-0.001222,0.000111
999996,1584371692708076544,0.286326,-33.246557,63.400569,182.943438,-7.269618,-17.111226,0.007577,-0.015732,0.000335,-0.002431,-0.005760,0.000127
999997,1584376022035134592,0.515196,22.353831,63.488923,182.585973,10.771528,5.881404,0.011293,0.047426,0.000262,0.005628,0.003599,0.000111
999998,1584378770814182656,0.610604,-74.945481,63.485937,183.079625,13.670553,-15.102476,0.014972,-0.025172,0.000284,-0.002339,0.002844,0.000116


In [22]:
a1= np.array([Filter.iloc[18218]])
a2= np.array([Filter.iloc[150139]])
a3= np.array([Filter.iloc[337522]])

A= np.concatenate((a1,a2,a3))
A

array([[ 8.04437110e+16,  6.32851972e-01,  4.62911479e+01,
         1.75793645e+01,  3.22362309e+01, -2.43459280e-01,
        -3.66843842e+00,  4.17568483e-02,  9.35016094e-03,
         2.86822495e-03, -3.10339163e-02, -5.19205955e-01,
         1.61595357e-03],
       [ 4.41538104e+17,  8.24493939e-01,  1.95185059e+01,
         4.95808345e+01,  5.31452126e+01, -6.05911192e+00,
        -2.54367173e+00,  1.82209001e-02,  9.26230245e-02,
         2.60949100e-04, -1.42132361e-02, -7.68595595e-03,
         3.09959774e-04],
       [ 1.09156323e+18,  2.17904574e-01,  3.42558735e+01,
         6.36758356e+01,  1.25162012e+02, -4.65385457e+01,
        -2.17193338e+01,  7.34858651e-03,  3.66470722e-02,
         3.68777466e-04, -2.25171154e-03, -8.29813725e-04,
         1.56979411e-04]])

In [23]:
A[0]

array([ 8.04437110e+16,  6.32851972e-01,  4.62911479e+01,  1.75793645e+01,
        3.22362309e+01, -2.43459280e-01, -3.66843842e+00,  4.17568483e-02,
        9.35016094e-03,  2.86822495e-03, -3.10339163e-02, -5.19205955e-01,
        1.61595357e-03])

In [24]:
o= Orbit(A[:,1:], radec=True)

In [25]:
Path(A, MWPotential2014, 1000, 1001)

,source_id,min_dist
0,8.044371e+16,17579.364496
1,4.415381e+17,49580.834493
2,1.091563e+18,63675.835573


What is left is for me to find the relative frequency and in all three dimensions and find the ones that are between 60-100myr.